# Strands Agents with AgentCore Memory (Long term memory via Tool)

## Introduction

This notebook demonstrates how to implement a **multi-agent system with shared long-term memory** using AgentCore Memory and the Strands framework. We will explore how multiple specialized agents can work together while accessing a common long-term memory store with dedicated namespaces for each agent.

### Tutorial Details

| Information         | Details                                                                          |
|:--------------------|:---------------------------------------------------------------------------------|
| Tutorial type       | Long term Conversational                                                         |
| Agent type          | Travel Booking Assistant                                                         |
| Agentic Framework   | Strands Agents                                                                   |
| LLM model           | Anthropic Claude Haiku 4.5                                                      |
| Tutorial components | AgentCore User Preferences Memory Extraction, Tool for storing and retrieving Memory              |
| Example complexity  | Intermediate                                                                     |

You will learn:

- How to set up a shared memory resource with a long-term memory strategy
- Creating specialized agents with access to their own memory namespaces
- Implementing a coordinator agent that delegates to specialized agents
- Leveraging structured memory namespaces for agent specialization

## Scenario context

In this example, we'll create a **Travel Planning System** with:
1. A Flight Booking Assistant with long-term memory of travel preferences and history
2. A Hotel Booking Assistant with long-term memory of accommodation preferences
3. A Travel Coordinator that orchestrates these specialized agents

Each specialized agent will access its own namespace within a common memory store, enabling them to build persistent understanding of user preferences over time. This approach demonstrates how complex domains can be broken down into specialized agents that share a memory infrastructure but maintain their own areas of expertise.

## Architecture
<div style="text-align:left">
    <img src="architecture.png" width="65%" />
</div>

## Prerequisites

To execute this tutorial you will need:
- Python 3.10+
- AWS credentials with Amazon Bedrock AgentCore Memory permissions
- Amazon Bedrock AgentCore SDK

Let's get started by setting up our environment and creating our shared long-term memory resource!

## Step 1: Environment set up
Let's begin importing all the necessary libraries and defining the clients to make this notebook work.

In [1]:
!pip install -qr requirements.txt

In [2]:
import logging
import time
from datetime import datetime
from strands.hooks import AfterInvocationEvent, HookProvider, HookRegistry

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s", datefmt="%Y-%m-%d %H:%M:%S")
logger = logging.getLogger("travel-assistant")

Define the region with the appropiate permissions for Amazon Bedrock models and AgentCore

In [3]:
region = "us-west-2" # Replace with your AWS region
MODEL_ID= "global.anthropic.claude-haiku-4-5-20251001-v1:0"

## Step 2: Creating Shared Memory Resource
In this section, we'll create common long-term memory store with dedicated namespaces for each agent.

In [4]:
from bedrock_agentcore.memory import MemoryClient
from bedrock_agentcore.memory.constants import StrategyType

client = MemoryClient(region_name=region)
memory_name = "TravelAgent_LTM"
memory_id = None

2025-12-13 19:45:57 - INFO - Initialized MemoryClient for control plane: us-west-2, data plane: us-west-2


In [5]:
from botocore.exceptions import ClientError

try:
    print("Creating Memory with Long-Term Strategy...")
    # Create the memory resource with a single long-term memory strategy
    # The {actorId} placeholder will be dynamically replaced with the actual actor ID
    memory = client.create_memory_and_wait(
        name=memory_name,
        description="Travel Agent with Long-Term Memory",
        strategies=[{
            StrategyType.USER_PREFERENCE.value: {
                "name": "UserPreferences",
                "description": "Captures user preferences",
                "namespaces": ["travel/{actorId}/preferences"]
            }
        }],
        event_expiry_days=7,  # Short-term conversation expires after 7 days
        max_wait=300,
        poll_interval=10
    )

    # Extract and print the memory ID
    memory_id = memory['id']
    print(f"Memory created successfully with ID: {memory_id}")
except ClientError as e:
    if e.response['Error']['Code'] == 'ValidationException' and "already exists" in str(e):
        # If memory already exists, retrieve its ID
        memories = client.list_memories()
        memory_id = next((m['id'] for m in memories if m['id'].startswith(memory_name)), None)
        logger.info(f"Memory already exists. Using existing memory ID: {memory_id}")
except Exception as e:
    # Handle any errors during memory creation
    logger.info(f"❌ ERROR: {e}")
    import traceback
    traceback.print_exc()
    # Cleanup on error - delete the memory if it was partially created
    if memory_id:
        try:
            client.delete_memory_and_wait(memory_id=memory_id)
            logger.info(f"Cleaned up memory: {memory_id}")
        except Exception as cleanup_error:
            logger.info(f"Failed to clean up memory: {cleanup_error}")

Creating Memory with Long-Term Strategy...


2025-12-13 19:46:09 - INFO - Created memory: TravelAgent_LTM-Zzv6NFD2v4
2025-12-13 19:46:09 - INFO - Created memory TravelAgent_LTM-Zzv6NFD2v4, waiting for ACTIVE status...
2025-12-13 19:48:52 - INFO - Memory TravelAgent_LTM-Zzv6NFD2v4 is now ACTIVE (took 162 seconds)


Memory created successfully with ID: TravelAgent_LTM-Zzv6NFD2v4


### Understanding Long-Term Memory Strategy

The memory resource we're creating uses AgentCore Memory's long-term memory capabilities with a single user preference strategy:

1. **User Preference Memory Strategy**: Automatically extracts and consolidates user preferences mentioned in conversations
2. **Actor-based Namespaces**: Uses the actor ID in the namespace path to create separate spaces for each agent
3. **Memory Persistence**: Unlike short-term memory that expires, extracted preferences persist beyond conversation expiry

With the namespace pattern `travel/{actorId}/preferences`, each specialized agent will have its own unique namespace based on its actor ID:
- Flight agent will access: `travel/flight-user-TIMESTAMP/preferences`
- Hotel agent will access: `travel/hotel-user-TIMESTAMP/preferences`

This allows each agent to maintain its own specialized knowledge while using a common memory infrastructure.

### Setting Up Agent Identities

In [6]:
# Create unique actor IDs for each specialized agent but share the session ID
flight_actor_id = f"flight-user-{datetime.now().strftime('%Y%m%d%H%M%S')}"
hotel_actor_id = f"hotel-user-{datetime.now().strftime('%Y%m%d%H%M%S')}"
session_id = f"travel-session-{datetime.now().strftime('%Y%m%d%H%M%S')}"
flight_namespace = f"travel/{flight_actor_id}/preferences"
hotel_namespace = f"travel/{hotel_actor_id}/preferences"

In [7]:
# Import the necessary components
from strands import Agent, tool
from strands_tools.agent_core_memory import AgentCoreMemoryToolProvider

### Step 3: Create Memory Hook Provider

This step defines our custom `MemoryHookProvider` class that automates memory operations. Hooks are special functions that run at specific points in an agent's execution lifecycle. The memory hook we're creating serves one primary function:

1. **Save Memories**: Stores new conversations after the agent responds

This creates a seamless memory experience without manual management.

In [8]:
class MemoryHookProvider(HookProvider):
    """Hook provider for automatic memory management"""
    
    def __init__(self, memory_id: str, client: MemoryClient):
        self.memory_id = memory_id
        self.client = client
    
    def save_memories(self, event: AfterInvocationEvent):
        """Save conversation after agent response"""
        try:
            messages = event.agent.messages
            if len(messages) >= 2:
                # Get last user and assistant messages
                user_msg = None
                assistant_msg = None
                
                for msg in reversed(messages):
                    if msg["role"] == "assistant" and not assistant_msg:
                        assistant_msg = msg["content"][0]["text"]
                    elif msg["role"] == "user" and not user_msg and "toolResult" not in msg["content"][0]:
                        user_msg = msg["content"][0]["text"]
                        break
                
                if user_msg and assistant_msg:
                    # Get session info from agent state
                    actor_id = event.agent.state.get("actor_id")
                    session_id = event.agent.state.get("session_id")
                    
                    if not actor_id or not session_id:
                        logger.warning("Missing actor_id or session_id in agent state")
                        return
                    
                    # Save conversation
                    self.client.create_event(
                        memory_id=self.memory_id,
                        actor_id=actor_id,
                        session_id=session_id,
                        messages=[(user_msg, "USER"), (assistant_msg, "ASSISTANT")]
                    )
                    logger.info("Saved conversation to memory")
                    
        except Exception as e:
            logger.error(f"Failed to save memories: {e}")
    
    def register_hooks(self, registry: HookRegistry) -> None:
        """Register memory hooks"""
        registry.add_callback(AfterInvocationEvent, self.save_memories)
        logger.info("Memory hooks registered")

### Creating Specialized Agents with Memory Access

Next, we'll define system prompts for our specialized agents.

In [9]:
# System prompt for the hotel booking specialist
HOTEL_BOOKING_PROMPT = f"""You are a hotel booking assistant. Help customers find hotels, make reservations, and answer questions about accommodations and amenities. 
Provide clear information about availability, pricing, and booking procedures in a friendly, helpful manner.Ask max two questions per turn. Keep the messages short, don't overwhelm the customer."""

# System prompt for the flight booking specialist
FLIGHT_BOOKING_PROMPT = f"""You are a flight booking assistant. Help customers find flights, make reservations, and answer questions about airlines, routes, and travel policies. 
Provide clear information about flight availability, pricing, schedules, and booking procedures in a friendly, helpful manner.Ask max two questions per turn. Keep the messages short, don't overwhelm the customer."""

### Implementing Agent Tools
Now we'll implement our specialized agents as tools that can be used by the coordinator agent:

In [10]:
@tool
def flight_booking_assistant(query: str) -> str:
    """
    Process and respond to flight booking queries.

    Args:
        query: A flight-related question about bookings, schedules, airlines, or travel policies

    Returns:
        Detailed flight information, booking options, or travel advice
    """
    try:
        provider_flight = AgentCoreMemoryToolProvider(
            memory_id=memory_id,      # Required
            actor_id=flight_actor_id, # Required
            session_id=session_id,    # Required
            region = region,
            namespace=flight_namespace
        )
        
        flight_memory_hooks = MemoryHookProvider(memory_id, client)

        flight_agent = Agent(
            tools=provider_flight.tools,
            hooks=[flight_memory_hooks],
            model=MODEL_ID,
            system_prompt=FLIGHT_BOOKING_PROMPT,
            state={"actor_id": flight_actor_id, "session_id": session_id}
        )

        # Call the agent and return its response
        response = flight_agent(query)
        return str(response)
    except Exception as e:
        return f"Error in flight booking assistant: {str(e)}"

@tool
def hotel_booking_assistant(query: str) -> str:
    """
    Process and respond to hotel booking queries.

    Args:
        query: A hotel-related question about accommodations, amenities, or reservations

    Returns:
        Detailed hotel information, booking options, or accommodation advice
    """
    try:
        provider_hotel = AgentCoreMemoryToolProvider(
            memory_id=memory_id,      
            actor_id=hotel_actor_id, 
            session_id=session_id,   
            region = region,
            namespace=hotel_namespace
        )

        hotel_memory_hooks = MemoryHookProvider(memory_id, client)

        hotel_booking_agent = Agent(
            tools=provider_hotel.tools,
            hooks=[hotel_memory_hooks],
            model=MODEL_ID,
            system_prompt=HOTEL_BOOKING_PROMPT,
            state={"actor_id": hotel_actor_id, "session_id": session_id}
        )
        
        # Call the agent and return its response
        response = hotel_booking_agent(query)
        return str(response)
    except Exception as e:
        return f"Error in hotel booking assistant: {str(e)}"

### Creating the Coordinator Agent

Finally, we'll create the main travel planning agent that coordinates between these specialized tools:

In [11]:
# System prompt for the coordinator agent
TRAVEL_AGENT_SYSTEM_PROMPT = """
You are a comprehensive travel planning assistant that coordinates between specialized tools:
- For flight-related queries (bookings, schedules, airlines, routes) → Use the flight_booking_assistant tool
- For hotel-related queries (accommodations, amenities, reservations) → Use the hotel_booking_assistant tool
- For complete travel packages → Use both tools as needed to provide comprehensive information
- For general travel advice or simple travel questions → Answer directly

Each agent will have its own memory in case the user asks about historic data.
When handling complex travel requests, coordinate information from both tools to create a cohesive travel plan.
Provide clear organization when presenting information from multiple sources. \
Ask max two questions per turn. Keep the messages short, don't overwhelm the customer.
"""

In [12]:
travel_agent = Agent(
    system_prompt=TRAVEL_AGENT_SYSTEM_PROMPT,
    model=MODEL_ID,
    tools=[flight_booking_assistant, hotel_booking_assistant]
)

## Testing the Multi-Agent Memory System

Let's test our multi-agent system with a travel planning scenario:

### Optional: Hydrate the long term memory of the flight booking assistant
Uncomment the following cell if you want to hydrate the flight booking agent.

In [13]:
"""flight_previous_messages = [
    ("Hi, I'm Sarah", "USER"),
    ("Hello Sarah! Welcome to FlightAssist. How can I help you with your travel plans today?", "ASSISTANT"),
    ("I'm looking to book a flight from New York to London sometime next month.", "USER"),
    ("I'd be happy to help you find flights from New York to London for next month. Do you have specific dates in mind, or are you flexible?", "ASSISTANT"),
    ("I'm thinking around the 15th to the 25th, but I can be a bit flexible.", "USER"),
    ("Great! That gives us some room to find the best options. Do you have any preferences regarding airlines or flight times?", "ASSISTANT"),
    ("I definitely prefer direct flights if possible. I really don't like layovers.", "USER"),
    ("I completely understand your preference for direct flights. Layovers can be inconvenient. Fortunately, there are several airlines offering direct flights between New York and London, including British Airways, American Airlines, Delta, and Virgin Atlantic.", "ASSISTANT"),
    ("That's good to hear. I've had good experiences with British Airways in the past.", "USER"),
    ("British Airways does offer excellent service on transatlantic routes. I'll keep that in mind when searching for options. Do you have any seating preferences or other requirements for your flight?", "ASSISTANT"),
    ("I always try to get an aisle seat. I like being able to get up without disturbing others, especially on long flights.", "USER"),
    ("An aisle seat is a great choice for long-haul flights like New York to London. I'll note your preference for aisle seating. Would you prefer to fly in the morning, afternoon, or evening?", "ASSISTANT"),
    ("I prefer overnight flights for long journeys. It helps me adjust to the time difference better.", "USER"),
    ("Overnight flights are indeed a smart choice for eastbound transatlantic travel. They allow you to arrive in London in the morning and help minimize jet lag. British Airways, Delta, and American all offer evening departures from New York that arrive in London the next morning.", "ASSISTANT"),
    ("Perfect! And I'm also wondering about baggage allowances since I'll be staying for about a week.", "USER"),
    ("For a week-long trip, most travelers find that a standard checked bag plus a carry-on is sufficient. British Airways typically allows one free checked bag on transatlantic flights in economy class, plus a carry-on and personal item. Would you like me to check the specific allowances for your preferred dates?", "ASSISTANT")
]

print("\nHydrating memories with previous conversations...")

# Save the conversation history to short-term memory
initial = client.create_event(
    memory_id=memory_id,
    actor_id=flight_actor_id,
    session_id=session_id,
    messages=flight_previous_messages,
)
print("✓ Conversation saved in short term memory")"""

'flight_previous_messages = [\n    ("Hi, I\'m Sarah", "USER"),\n    ("Hello Sarah! Welcome to FlightAssist. How can I help you with your travel plans today?", "ASSISTANT"),\n    ("I\'m looking to book a flight from New York to London sometime next month.", "USER"),\n    ("I\'d be happy to help you find flights from New York to London for next month. Do you have specific dates in mind, or are you flexible?", "ASSISTANT"),\n    ("I\'m thinking around the 15th to the 25th, but I can be a bit flexible.", "USER"),\n    ("Great! That gives us some room to find the best options. Do you have any preferences regarding airlines or flight times?", "ASSISTANT"),\n    ("I definitely prefer direct flights if possible. I really don\'t like layovers.", "USER"),\n    ("I completely understand your preference for direct flights. Layovers can be inconvenient. Fortunately, there are several airlines offering direct flights between New York and London, including British Airways, American Airlines, Delta, a

In [14]:
travel_agent("Hello, I would like to book a trip from Copenhagen to Delhi. From July 1 to August 2.")

2025-12-13 19:50:43 - INFO - Creating Strands MetricsClient


I'd be happy to help you plan your trip from Copenhagen to Delhi! To get you the best options for flights and accommodations, I need a couple of quick details:

1. **How many passengers** will be traveling?
2. **Do you have any specific hotel preferences** (e.g., budget range, specific area of Delhi, amenities)?

Once I have these details, I can search for both flights and hotels for your July 1 - August 2 trip.

AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': "I'd be happy to help you plan your trip from Copenhagen to Delhi! To get you the best options for flights and accommodations, I need a couple of quick details:\n\n1. **How many passengers** will be traveling?\n2. **Do you have any specific hotel preferences** (e.g., budget range, specific area of Delhi, amenities)?\n\nOnce I have these details, I can search for both flights and hotels for your July 1 - August 2 trip."}]}, metrics=EventLoopMetrics(cycle_count=1, tool_metrics={}, cycle_durations=[1.6560142040252686], traces=[<strands.telemetry.metrics.Trace object at 0x7f111ecca480>], accumulated_usage={'inputTokens': 890, 'outputTokens': 106, 'totalTokens': 996}, accumulated_metrics={'latencyMs': 1636}), state={}, interrupts=None, structured_output=None)

In [15]:
travel_agent("I prefer direct flights")


Tool #1: flight_booking_assistant


2025-12-13 19:50:51 - INFO - Memory hooks registered


I'd be happy to help you search for flights from Copenhagen to Delhi! 

However, I need to let you know that I'm a flight booking assistant, but I don't have access to a live flight search system or real-time flight database. To find actual flights matching your criteria, you would need to:

**Your Trip Details:**
- **From:** Copenhagen (CPH)
- **To:** Delhi (DEL)
- **Departure:** July 1
- **Return:** August 2
- **Type:** Direct flights

**Recommended booking platforms:**
- Skyscanner.com
- Kayak.com
- Google Flights
- Airline websites (SAS, Emirates, etc.)

**Things to know about this route:**
- Direct flights from Copenhagen to Delhi are typically operated by airlines like SAS, Emirates, or Air India
- Flight duration is usually around 7-8 hours
- This is a popular summer travel period, so booking early is recommended
- Prices tend to be higher during peak season

Would you like me to help you with information about specific airlines, baggage policies, visa requirements for India, or

2025-12-13 19:50:54 - INFO - Created event: 0000001765655454303#05383325
2025-12-13 19:50:54 - INFO - Saved conversation to memory


Great! For your trip from Copenhagen to Delhi (July 1 - August 2), here's what you need to know about **direct flights**:

**Key Information:**
- Direct flights on this route are typically around **7-8 hours**
- Airlines offering this route include SAS, Emirates, and Air India
- Peak summer travel means **booking early is recommended**

**To find and book:**
- Skyscanner, Google Flights, Kayak, or airline websites
- This is a popular route, so prices will be higher in July

Now, for your **hotel in Delhi** - could you tell me:
1. **How many people** will be staying?
2. **Budget preference** for accommodation (budget, mid-range, luxury)?

I can search for hotel options while you look at flights!

AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': "Great! For your trip from Copenhagen to Delhi (July 1 - August 2), here's what you need to know about **direct flights**:\n\n**Key Information:**\n- Direct flights on this route are typically around **7-8 hours**\n- Airlines offering this route include SAS, Emirates, and Air India\n- Peak summer travel means **booking early is recommended**\n\n**To find and book:**\n- Skyscanner, Google Flights, Kayak, or airline websites\n- This is a popular route, so prices will be higher in July\n\nNow, for your **hotel in Delhi** - could you tell me:\n1. **How many people** will be staying?\n2. **Budget preference** for accommodation (budget, mid-range, luxury)?\n\nI can search for hotel options while you look at flights!"}]}, metrics=EventLoopMetrics(cycle_count=3, tool_metrics={'flight_booking_assistant': ToolMetrics(tool={'toolUseId': 'tooluse_mjp8fAQyQOOjLLq5gZwE5Q', 'name': 'flight_booking_assistant', 'inpu

In [16]:
travel_agent("I would like a flight in the evening, in economy")


Tool #2: flight_booking_assistant


2025-12-13 19:51:08 - INFO - Memory hooks registered


I'd be happy to help you find flights from Copenhagen to Delhi! However, I need to clarify a couple of things:

1. **Are you looking for a one-way flight or a round-trip?** (I notice you mentioned July 1 to August 2 - is that outbound and return dates, or just the travel period?)

2. **For evening departures** - what time range works best for you? (For example, after 5 PM, after 6 PM, etc.)

Once I have these details, I can search for available direct flights for one passenger in economy class! 

2025-12-13 19:51:10 - INFO - Created event: 0000001765655470818#f60462c6
2025-12-13 19:51:10 - INFO - Saved conversation to memory


✈️Thanks for those details! To complete your flight search, I need one clarification:

**For your evening departure preference** - what time works best for you? (e.g., after 5 PM, after 6 PM, late evening?)

Also, just to confirm - is this a **round-trip** with return on August 2, or a one-way flight departing July 1?

AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': 'Thanks for those details! To complete your flight search, I need one clarification:\n\n**For your evening departure preference** - what time works best for you? (e.g., after 5 PM, after 6 PM, late evening?)\n\nAlso, just to confirm - is this a **round-trip** with return on August 2, or a one-way flight departing July 1?'}]}, metrics=EventLoopMetrics(cycle_count=5, tool_metrics={'flight_booking_assistant': ToolMetrics(tool={'toolUseId': 'tooluse_IOj0mH6gTrC0fwyyoUDeZQ', 'name': 'flight_booking_assistant', 'input': {'query': 'Evening direct flights from Copenhagen to Delhi, July 1 to August 2, economy class, one passenger'}}, call_count=2, success_count=2, error_count=0, total_time=5.7265706062316895)}, cycle_durations=[1.6560142040252686, 2.43491530418396, 2.088308572769165], traces=[<strands.telemetry.metrics.Trace object at 0x7f111ecca480>, <strands.telemetry.metrics.Trace object at 0x7f111ecca990>

In [17]:
travel_agent("I would like to fly from Copenhagen, and return 15 days later")

Perfect! So that's:
- **Outbound:** July 1 (evening departure)
- **Return:** July 16
- **Class:** Economy
- **Passengers:** 1
Tool #3: flight_booking_assistant

Tool #4: hotel_booking_assistant


2025-12-13 19:51:47 - INFO - Memory hooks registered
2025-12-13 19:51:47 - INFO - Memory hooks registered


I appreciate you providing those details! However, I need to let you know that as a flight booking assistant, I don't have access to aI'd be happy to help you find a hotel in Delhi for your 15-night stay (July 1-16) live flight database or booking system to search real-time availability and prices.

To help you find these for 1 guest!

To give you the best recommendations, I flights, I recommend:

1. **Check directly with have a couple of quick questions:

1. **What's your budget range?** (e.g., budget airlines** serving this route like:
   - SAS (Scandinavian Airlines)
   - Lufthansa, mid-range, luxury)
   - Air India
   - Emirates
   - Qatar
2. **Any specific preferences?** (e.g., amenities Airways

2. **Use flight search engines** like:
   - Google Flights
   - Skyscanner
   - Kayak like gym, pool, restaurant, free WiF
   - Kiwi.com
   - Expi, etc.)

This will help me narrow down the best options for you!edia

Let me save your preferences so I can help you with other questions:
Tool

2025-12-13 19:51:50 - INFO - Created event: 0000001765655510225#83fcdf8e
2025-12-13 19:51:50 - INFO - Saved conversation to memory


**Can I help you with:**
- General questions about airlines, routes, or baggage policies?
- Tips for getting the best prices on this route?

2025-12-13 19:51:52 - INFO - Created event: 0000001765655512136#423ba746
2025-12-13 19:51:52 - INFO - Saved conversation to memory


Great! I've got your trip details set. For **hotels in Delhi** (July 1-16, 15 nights for 1 guest):

What's your **budget preference** for accommodation? (budget, mid-range, or luxury)

AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': "Great! I've got your trip details set. For **hotels in Delhi** (July 1-16, 15 nights for 1 guest):\n\nWhat's your **budget preference** for accommodation? (budget, mid-range, or luxury)"}]}, metrics=EventLoopMetrics(cycle_count=7, tool_metrics={'flight_booking_assistant': ToolMetrics(tool={'toolUseId': 'tooluse_GvjF4jeATU6VpDsgenuyYw', 'name': 'flight_booking_assistant', 'input': {'query': 'Round-trip direct flights from Copenhagen to Delhi, departing July 1 in the evening, returning July 16, economy class, 1 passenger'}}, call_count=3, success_count=3, error_count=0, total_time=10.47775912284851), 'hotel_booking_assistant': ToolMetrics(tool={'toolUseId': 'tooluse_33PAElD5SB6CiMvqAjzJaA', 'name': 'hotel_booking_assistant', 'input': {'query': 'Hotel accommodations in Delhi for 15 nights, July 1-16, 1 guest'}}, call_count=1, success_count=1, error_count=0, total_time=2.872657537460327)}, cycle_duratio

## Testing Memory Persistence

To test if our memory system is working correctly, we'll create a new instance of the travel agent and see if it can access the previously stored information:

In [18]:
time.sleep(60) # Let's give the memory some time to process the events..
# Create a new instance of the travel agent
new_travel_agent = Agent(
    system_prompt=TRAVEL_AGENT_SYSTEM_PROMPT,
    model=MODEL_ID,
    tools=[flight_booking_assistant, hotel_booking_assistant]
)

# Ask about previous conversations
new_travel_agent("Can you remind me about flight preferences?")

I don't have access to your previous conversation history or stored flight preferences. Each conversation starts fresh for me.

However, I can help you set up your flight preferences now! To assist you better, could you tell me:

1. **What are your typical flight preferences?** (e.g., preferred airlines, seat types like aisle/window, direct flights only, etc.)
2. **What's your travel style?** (e.g., budget-conscious, prefer early morning departures, need flexibility, etc.)

Once you share these details, I can keep them in mind for this conversation and help you find flights that match what you're looking for!

AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': "I don't have access to your previous conversation history or stored flight preferences. Each conversation starts fresh for me.\n\nHowever, I can help you set up your flight preferences now! To assist you better, could you tell me:\n\n1. **What are your typical flight preferences?** (e.g., preferred airlines, seat types like aisle/window, direct flights only, etc.)\n2. **What's your travel style?** (e.g., budget-conscious, prefer early morning departures, need flexibility, etc.)\n\nOnce you share these details, I can keep them in mind for this conversation and help you find flights that match what you're looking for!"}]}, metrics=EventLoopMetrics(cycle_count=1, tool_metrics={}, cycle_durations=[2.5693137645721436], traces=[<strands.telemetry.metrics.Trace object at 0x7f111dc777d0>], accumulated_usage={'inputTokens': 874, 'outputTokens': 143, 'totalTokens': 1017}, accumulated_metrics={'latencyMs': 254

## Summary

In this notebook, we've demonstrated:

1. How to create a shared memory resource for multiple agents
2. How to implement specialized agents as tools with memory access
3. How to coordinate between multiple agents while maintaining conversation context
4. How memory persists across different agent instances

This multi-agent architecture with shared memory provides a powerful approach for building complex conversational AI systems that can handle specialized domains while maintaining a cohesive user experience.

## Clean up
Let's delete the memory to clean up the resources used in this notebook.

In [ ]:
# client.delete_memory_and_wait(
#        memory_id = memory_id,
#        max_wait = 300,
#        poll_interval =10
#)